## Step 6 – Deploy Model - Task 1. Scoring Engine


###Load Libraries


In [1]:
!pip install sklearn --upgrade

In [2]:
!pip install skl2onnx

In [3]:
!pip install onnxruntime

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt

/Users/asathi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/Users/asathi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=

### Set up environment and connect to Google Drive

In [5]:
using_Google_colab = False
using_Anaconda_on_Mac_or_Linux = True
using_Anaconda_on_windows = False

In [6]:
if using_Google_colab:
    from google.colab import drive
    drive.mount('/content/drive')

### Upload intermediate classification modeling data for LA County
(see Step-4-Develop-Model-Task-4-Classification notebook for details)

## DM6.1 Open Notebook and upload the LA county merged data - Activity 1


In [7]:
if using_Google_colab:
  merged_LA = pd.read_csv('/content/drive/MyDrive/COVID_Project/output/merged_LA.csv')
if using_Anaconda_on_Mac_or_Linux:
  merged_LA = pd.read_csv('../output/merged_LA.csv')
if using_Anaconda_on_windows:
  merged_LA = pd.read_csv(r'..\output\merged_LA.csv')
merged_LA

,Unnamed: 0,countyFIPS,Date,Incremental Cases,population,Month,Case Change,retail_recreation,grocery_pharmacy,parks,...,workplaces,residential,Original Month,delta_retail_recreation,delta_grocery_pharmacy,delta_parks,delta_transit,delta_workplaces,delta_residential,case_direction
0,2,6037,2020-04-30,672.366667,10039107,2020-04-30,575.269892,-21.064516,-1.806452,-19.903226,...,-21.483871,10.935484,2020-03-31,-23.131183,-2.473118,-31.836559,-27.417204,-21.150538,10.935484,1
1,3,6037,2020-05-31,1026.258065,10039107,2020-05-31,353.891398,-51.766667,-22.066667,-44.900000,...,-48.566667,22.166667,2020-04-30,-30.702151,-20.260215,-24.996774,-25.116129,-27.082796,11.231183,1
2,4,6037,2020-06-30,1617.766667,10039107,2020-06-30,591.508602,-45.741935,-12.096774,-18.806452,...,-40.709677,17.580645,2020-05-31,6.024731,9.969892,26.093548,8.632258,7.856989,-4.586022,1
3,5,6037,2020-07-31,2740.387097,10039107,2020-07-31,1122.620430,-34.033333,-10.466667,-8.866667,...,-34.500000,13.833333,2020-06-30,11.708602,1.630108,9.939785,5.434409,6.209677,-3.747312,1
4,6,6037,2020-08-31,1718.935484,10039107,2020-08-31,-1021.451613,-32.096774,-10.483871,-6.935484,...,-37.580645,13.677419,2020-07-31,1.936559,-0.017204,1.931183,-1.563441,-3.080645,-0.155914,-1
5,7,6037,2020-09-30,951.033333,10039107,2020-09-30,-767.902151,-31.290323,-12.129032,-7.516129,...,-35.322581,12.129032,2020-08-31,0.806452,-1.645161,-0.580645,1.129032,2.258065,-1.548387,-1
6,8,6037,2020-10-31,1203.838710,10039107,2020-10-31,252.805376,-30.500000,-13.533333,-16.766667,...,-35.900000,12.066667,2020-09-30,0.790323,-1.404301,-9.250538,-1.198925,-0.577419,-0.062366,1
7,9,6037,2020-11-30,3110.033333,10039107,2020-11-30,1906.194624,-29.677419,-12.806452,-19.709677,...,-32.290323,11.129032,2020-10-31,0.822581,0.726882,-2.943011,3.456989,3.609677,-0.937634,1
8,10,6037,2020-12-31,11137.550000,10039107,2020-12-31,8027.516667,-31.300000,-13.933333,-27.733333,...,-34.866667,12.600000,2020-11-30,-1.622581,-1.126882,-8.023656,-5.590323,-2.576344,1.470968,1


## DM6.2 Prepare inputs and outputs for classification model and develop model - Activity 2


### Prepare and apply the model


In [8]:
y_LA = merged_LA['case_direction'].values
X_LA = merged_LA[['delta_retail_recreation', 
               'delta_grocery_pharmacy', 
               'delta_parks', 
               'delta_transit', 
               'delta_workplaces', 
               'delta_residential']].values
X_LA

array([[-2.31311828e+01, -2.47311828e+00, -3.18365591e+01,
        -2.74172043e+01, -2.11505376e+01,  1.09354839e+01],
       [-3.07021505e+01, -2.02602151e+01, -2.49967742e+01,
        -2.51161290e+01, -2.70827957e+01,  1.12311828e+01],
       [ 6.02473118e+00,  9.96989247e+00,  2.60935484e+01,
         8.63225806e+00,  7.85698925e+00, -4.58602151e+00],
       [ 1.17086022e+01,  1.63010753e+00,  9.93978495e+00,
         5.43440860e+00,  6.20967742e+00, -3.74731183e+00],
       [ 1.93655914e+00, -1.72043011e-02,  1.93118280e+00,
        -1.56344086e+00, -3.08064516e+00, -1.55913978e-01],
       [ 8.06451613e-01, -1.64516129e+00, -5.80645161e-01,
         1.12903226e+00,  2.25806452e+00, -1.54838710e+00],
       [ 7.90322581e-01, -1.40430108e+00, -9.25053763e+00,
        -1.19892473e+00, -5.77419355e-01, -6.23655914e-02],
       [ 8.22580645e-01,  7.26881720e-01, -2.94301075e+00,
         3.45698925e+00,  3.60967742e+00, -9.37634409e-01],
       [-1.62258065e+00, -1.12688172e+00, -8.023

In [9]:
y_LA

array([ 1,  1,  1,  1, -1, -1,  1,  1,  1])

In [10]:
decision_tree = DecisionTreeClassifier()
X_train_LA, X_test_LA, y_train_LA, y_test_LA = train_test_split(X_LA, 
                                                                y_LA, 
                                                                test_size=0.2, 
                                                                random_state=0)
model_LA = decision_tree.fit(X_train_LA, y_train_LA)

/Users/asathi/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)


## Save Test data into CSV file

In [11]:
df_test_y_LA = pd.DataFrame(y_test_LA, columns=['Y'])
df_test_y_LA

,Y
0,1
1,1


In [12]:
df_test_y_LA.to_csv('../output/test_y_LA')

In [13]:
df_test_X_LA = pd.DataFrame(X_test_LA, columns=['X1', 'X2', 'X3', 'X4', 'X5', 'X6'])
df_test_X_LA

,X1,X2,X3,X4,X5,X6
0,0.822581,0.726882,-2.943011,3.456989,3.609677,-0.937634
1,6.024731,9.969892,26.093548,8.632258,7.856989,-4.586022


In [14]:
df_test_X_LA.to_csv('../output/test_X_LA')

## DM6.3 - Pack and save model - Activity 3

In [15]:
# Convert into ONNX format

initial_type = [('float_input', FloatTensorType([None, 6]))]
onx = convert_sklearn(decision_tree, initial_types=initial_type)
with open("../output/rf_classification_LA.onnx", "wb") as f:
    f.write(onx.SerializeToString())

### This model can now be tested in Google Colab  - This is your Activity 4 and listed under Task 2 of  Scoring Engine notbook code

## DM6.5  Repeat the exercise for merged data for all counties - CLASS ASSIGNMENT


### Open notebook and load All county merged data

In [16]:
if using_Google_colab:
  merged_all = pd.read_csv('/content/drive/MyDrive/COVID_Project/output/merged_all.csv')
if using_Anaconda_on_Mac_or_Linux:
  merged_all = pd.read_csv('../output/merged_all.csv')
if using_Anaconda_on_windows:
  merged_all = pd.read_csv(r'..\output\merged_qll.csv')
merged_all

,Unnamed: 0,countyFIPS,Date,Incremental Cases,population,Month,Case Change,retail_recreation,grocery_pharmacy,parks,...,workplaces,residential,Original Month,delta_retail_recreation,delta_grocery_pharmacy,delta_parks,delta_transit,delta_workplaces,delta_residential,case_direction
0,0,1001,2020-02-29,0.000000,55869,2020-02-29,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2020-01-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1
1,1,1001,2020-03-31,0.225806,55869,2020-03-31,0.225806,4.733333,2.533333,-23.000000,...,0.266667,0.200000,2020-02-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
2,2,1001,2020-04-30,1.166667,55869,2020-04-30,0.940860,-6.354839,12.612903,-14.000000,...,-13.419355,6.090909,2020-03-31,-11.088172,10.079570,9.000000,0.000000,-13.686022,5.890909,1
3,3,1001,2020-05-31,5.741935,55869,2020-05-31,4.575269,-25.700000,4.033333,-49.500000,...,-37.833333,15.863636,2020-04-30,-19.345161,-8.579570,-35.500000,0.000000,-24.413978,9.772727,1
4,4,1001,2020-06-30,10.566667,55869,2020-06-30,4.824731,-3.677419,13.548387,0.000000,...,-27.451613,10.850000,2020-05-31,22.022581,9.515054,0.000000,0.000000,10.381720,-5.013636,1
5,5,1001,2020-07-31,15.419355,55869,2020-07-31,4.852688,-1.066667,11.133333,0.000000,...,-21.866667,8.409091,2020-06-30,2.610753,-2.415054,0.000000,0.000000,5.584946,-2.440909,1
6,6,1001,2020-08-31,13.774194,55869,2020-08-31,-1.645161,-2.516129,6.967742,0.000000,...,-26.483871,8.260870,2020-07-31,-1.449462,-4.165591,0.000000,0.000000,-4.617204,-0.148221,-1
7,7,1001,2020-09-30,11.633333,55869,2020-09-30,-2.140860,-3.000000,4.500000,0.000000,...,-24.037037,6.571429,2020-08-31,-0.483871,-2.467742,0.000000,0.000000,2.446834,-1.689441,-1
8,8,1001,2020-10-31,11.870968,55869,2020-10-31,0.237634,-6.800000,-2.350000,0.000000,...,-22.678571,4.695652,2020-09-30,-3.800000,-6.850000,0.000000,0.000000,1.358466,-1.875776,1
9,9,1001,2020-11-30,20.700000,55869,2020-11-30,8.829032,-2.645161,2.419355,-24.000000,...,-19.387097,3.322581,2020-10-31,4.154839,4.769355,0.000000,0.000000,3.291475,-1.373072,1


### Prepare and apply the model

In [17]:
y_all = merged_all['case_direction'].values
X_all = merged_all[['delta_retail_recreation', 
               'delta_grocery_pharmacy', 
               'delta_parks', 
               'delta_transit', 
               'delta_workplaces', 
               'delta_residential']].values
X_all

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-11.08817204,  10.07956989,   9.        ,   0.        ,
        -13.68602151,   5.89090909],
       ...,
       [  0.        ,   0.        ,   0.        ,   0.        ,
          9.2       ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.72727273,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        -10.82251082,   0.        ]])

In [18]:
y_all

array([-1,  1,  1, ...,  1,  1, -1])

In [20]:
decision_tree = DecisionTreeClassifier()
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, 
                                                                y_all, 
                                                                test_size=0.2, 
                                                                random_state=0)
model_all = decision_tree.fit(X_train_all, y_train_all)

/Users/asathi/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)


### Save all county test data into a csv file

In [21]:
df_test_y_all = pd.DataFrame(y_test_all, columns=['Y'])
df_test_y_all

,Y
0,1
1,1
2,1
3,1
4,1
5,-1
6,1
7,1
8,-1
9,-1


In [22]:
df_test_y_all.to_csv('../output/test_y_all')

In [23]:
df_test_X_all = pd.DataFrame(X_test_all, columns=['X1', 'X2', 'X3', 'X4', 'X5', 'X6'])
df_test_X_all

,X1,X2,X3,X4,X5,X6
0,0.456452,3.856452,0.000000,-2.419355,3.883257,-0.403927
1,3.737500,-3.012500,0.000000,0.000000,4.950000,-2.095238
2,-14.679644,-10.294624,0.000000,0.000000,-18.136559,6.090909
3,-27.664516,-16.838710,0.000000,-31.258065,-24.727957,9.675325
4,4.898190,9.200000,0.000000,5.733871,0.106452,-0.080214
5,-20.068817,-36.405376,-5.291667,0.000000,-25.890323,11.162879
6,0.000000,0.000000,0.000000,3.757895,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,30.587500,16.533333,0.000000,0.000000,11.436559,0.000000
9,-5.132258,-3.626882,-30.964516,-7.778495,-4.945161,2.612903


In [24]:
df_test_X_all.to_csv('../output/test_X_all')

### pack and save model

In [25]:
# Convert into ONNX format

initial_type = [('float_input', FloatTensorType([None, 6]))]
onx = convert_sklearn(decision_tree, initial_types=initial_type)
with open("../output/rf_classification_all.onnx", "wb") as f:
    f.write(onx.SerializeToString())